In [ ]:
%pip install msal requests 

In [ ]:
%pip install python-pptx   

In [ ]:
%pip install msal requests pandas openpyxl python-pptx 

In [ ]:
# Restart the Python kernel dbutils.library.restartPython() 

In [ ]:
import requests
import pandas as pd
from io import BytesIO
from pptx import Presentation  
import msal


In [ ]:
tenant_id = dbutils.secrets.get(scope="company", key="tenant_id")
client_id = dbutils.secrets.get(scope="company", key="client_id")
client_secret = dbutils.secrets.get(scope="company", key="client_secret_id")

Authentication MSAL

In [ ]:

authority = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
scope = ["https://graph.microsoft.com/.default"]

# Set up MSAL authentication
app = msal.ConfidentialClientApplication(
    client_id=client_id,
    client_credential=client_secret,
    authority=f"https://login.microsoftonline.com/{tenant_id}",
)

# Acquire token or refresh if expired
def get_access_token():
    token_response = app.acquire_token_silent(scopes=scope, account=None)
    
    # If no token is available, acquire a new one
    if not token_response:
        token_response = app.acquire_token_for_client(scopes=scope)

    access_token = token_response.get("access_token")
    if access_token:
        print("Access token acquired successfully.")
        return access_token
    else:
        raise Exception("Failed to acquire access token.")

# Get the token
access_token = get_access_token()

# Set headers with the access token
headers = {
    "Authorization": f"Bearer {access_token}"
}


In [ ]:
import urllib.parse

#Set up the sharepoint location with spaces and everythin and this cell code will encoded to be processed
# Define the SharePoint site and file path and encode it
sharepoint_site = 'andrey.sharepoint.com'
site_name = 'Production'  
file_path = '/Shared Documents/Documents/Volumes.xlsx' 
#just change the file_path, change the URL, you have to put it as it is, with spaces and slashes and dots.

encoded_file_path = urllib.parse.quote(file_path)
print(f"Encoded file path: {encoded_file_path}")

In [ ]:
# Step 1: Retrieve the Site ID we are going to map
site_url = f"https://graph.microsoft.com/v1.0/sites/{sharepoint_site}:/sites/{site_name}"
site_response = requests.get(site_url, headers=headers)

if site_response.status_code == 200:
    site_data = site_response.json()
    site_id = site_data['id']  # Extract the site ID
    print(f"Successfully accessed the SharePoint site with Site ID: {site_id}")
else:
    raise Exception(f"Failed to access SharePoint site: {site_response.status_code}, {site_response.text}")


In [ ]:
# Step 2: Retrieve the Document Library (Drive ID) this driver it's requested by microsoft graog in the endpoint
drive_url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives"
drive_response = requests.get(drive_url, headers=headers)

if drive_response.status_code == 200:
    drives_data = drive_response.json()
    drive_id = None
    for drive in drives_data['value']:
        if "Shared Documents" or "Documents" in drive['name']:  # Match against "Shared Documents"
            drive_id = drive['id']
            break
    
    if drive_id is None:
        raise Exception("Drive ID for 'Shared Documents' not found.")
    else:
        print(f"Drive ID: {drive_id}")
else:
    raise Exception(f"Failed to retrieve drives: {drive_response.status_code}, {drive_response.text}")


In [ ]:
# Search for the file ID using the search API
search_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/search(q='{file_path.split('/')[-1]}')"
search_response = requests.get(search_url, headers=headers)

if search_response.status_code == 200:
    search_data = search_response.json()
    if len(search_data['value']) > 0:
        file_id = search_data['value'][0]['id']  # Extract the first matching file's ID
        print(f"Successfully retrieved the File ID: {file_id}")
    else:
        raise Exception("File not found in search results.")
else:
    raise Exception(f"Failed to search for the file: {search_response.status_code}, {search_response.text}")


In [ ]:
#final endpoint
file_content_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/items/{file_id}/content"

Load the specific sheet from Sharepoint to a Dataframe

In [ ]:
# Function to load a specific sheet from the Excel file
def load_excel_from_sharepoint(file_url, headers, sheet_name=None):
    response = requests.get(file_url, headers=headers)
    if response.status_code == 200:
        file_stream = BytesIO(response.content)
        # Specify the sheet_name parameter to select a specific sheet
        df = pd.read_excel(file_stream, sheet_name=sheet_name)  
        return df
    else:
        raise Exception(f"Failed to access the file: {response.status_code}, {response.text}")

# Specify the name of the sheet you want to load
sheet_name = "Volume"  # Replace with the actual sheet name

# Load the specified sheet from the Excel file into a DataFrame
df_target_sheet = load_excel_from_sharepoint(file_content_url, headers, sheet_name=sheet_name)
print(f"DataFrame loaded from SharePoint (Sheet: {sheet_name}):")
df_target_sheet.head()

Transform your data if it's necessary

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp

# Initialize Spark session
spark = SparkSession.builder.appName("SaveDeltaTable").getOrCreate()

# Add a new column with the current timestamp as 'timeseries'
spark_df_final = spark_df_final.withColumn('snapshot_date', current_timestamp())

# Print the updated schema to verify the new column
print("\nSchema of the final Spark DataFrame with the 'timeseries' column:")
spark_df_final.printSchema()

# Display the updated Spark DataFrame
print("\nPreview of the final Spark DataFrame with 'timeseries':")
spark_df_final.show(20)


In [ ]:
# Define the database and table name in this case I'm using Databricks to create the table
database_name = "default.andrey"  # Replace with your database name if different
table_name = "volumenes"

# Ensure the database exists
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")

# Append the new data to the existing Delta table and enable schema evolution
spark_df_final.write.mode("append").format("delta") \
    .option("mergeSchema", "true") \
    .saveAsTable(f"{database_name}.{table_name}")

#If you are running this for the first time or want to overwrite the table use this code section
#spark_df_final.write.mode("append").format("delta") \
#    .option("mergeSchema", "true") \
#    .saveAsTable(f"{database_name}.{table_name}")

print(f"Data successfully appended to Delta table: {database_name}.{table_name}")

In [ ]:
%sql
SELECT * 
FROM default.andrey.volumenes